In [1]:
import numpy as np
from scipy.optimize import fsolve
from itertools import permutations
np.set_printoptions(suppress=True)

# Helper Functions

In [2]:
# sigmoid function
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

# derivative of sigmoid function
def sigmoid_prime(x):
  return np.exp(-x) / (1 + np.exp(-x))**2

In [3]:
def system_of_eqs(ridge_param, kappa, gamma, alpha, tau, lambda_):
    # This returns the system of equations involving alpha, sigma, and lambda
    
    # Defines the proximal function
    prox = lambda z : fsolve(lambda t  : lambda_ *  sigmoid(t) + t - z, z, xtol=1e-8)
    
    # We now compute the density of Q_1, Q_2 which we express as the density of
    # Z_1, Z_2 where Q_1 = gamma Z_1 and Q_2 = tau Z_2
    # The correlation between Z_1 and Z_2 is alpha x gamma / tau
    rho = alpha * gamma / tau
    
    # Set up a fine grid for computing the density
    density = lambda z1 ,z2 : np.multiply(1 / (2 * np.pi * np.sqrt(1 - rho ** 2)),np.exp(- (z1 ** 2 - np.multiply(2 * rho * z1,z2) + z2 ** 2) / 2 / (1 - rho ** 2)))
    step_size = 0.025
    z1 = np.arange(-8,8+step_size,step_size)
    z2 = np.arange(-8,8+step_size,step_size)
    Z1,Z2 = np.meshgrid(z1,z2)
    D = density(Z1,Z2)

    # Compute prox(Q_2) on a fine grid
    proxQ2 = prox(tau * z2)

    # Evaluate rho_prime and rho_double_prime at the prox
    sigmoid_proxQ2 = sigmoid(proxQ2)
    sigmoid_prime_proxQ2 = sigmoid_prime(proxQ2)

    # Helper functions for tensor and numerical integration
    make_tensor = lambda z1  ,z2  : np.outer(z1,z2)
    trapezoidal_rule = lambda X  : step_size ** 2 * sum(X.ravel())
    
    # Evaluate the equation's right-hand sides
    RHS1 = 1 / kappa * trapezoidal_rule(np.multiply(make_tensor(2 *  sigmoid(- gamma * z1),(lambda_ *  sigmoid_proxQ2) ** 2),D))
    RHS2 = trapezoidal_rule(np.multiply(make_tensor(np.multiply(- 2 *  sigmoid(- gamma * z1) * gamma,z1), sigmoid_proxQ2),D))
    RHS3 = trapezoidal_rule(np.multiply(make_tensor(2 *  sigmoid(- gamma * z1),1.0 / (1 + lambda_ * sigmoid_prime_proxQ2)),D))
    
    # Return LHS - RHS for the equations
    return tau ** 2 - alpha ** 2 * gamma ** 2 - RHS1, alpha * gamma ** 2 * ridge_param - RHS2, 1 - kappa + lambda_ * ridge_param - RHS3

def solve_eqs(kappa,gamma,ridge_param): 
    # This solves the system of equations

    # Initial guess for alpha, lambda, and tau
    alpha0, lambda0 = 1.2, 1.2
    tau0 = np.sqrt(2) * alpha0 * gamma
    x0 = [alpha0, tau0, lambda0]

    # Solve the system of equations using fsolve
    x = fsolve(lambda x: system_of_eqs(ridge_param,kappa,gamma,x[0],x[1],x[2]), x0, xtol=1e-8)
    alpha_star, tau_star, lambda_star = x[0], x[1], x[2]

    # Compute and return the final values of alpha_star, sigma_star, and lambda_star
    sigma_star = np.sqrt((tau_star ** 2 - alpha_star ** 2 * gamma ** 2) / kappa)
    alpha_star = alpha_star * kappa / (kappa - lambda_star * ridge_param)
    sigma_star = sigma_star * kappa / (kappa - lambda_star * ridge_param)
    return alpha_star, sigma_star, lambda_star

# Define Parameters

In [4]:
n, p = 10000, 700
r1, r2= 1/3, 1/3
n1, n2 = int(n*r1), int(n*r2)
n3 = n - n1 - n2
r1, r2, r3 = n1/n, n2/n, n3/n
alpha1, alpha0 = 2, 0
gamma = 0.1

kappa, kappa1, kappa2, kappa3 = p/n, p/n1, p/n2, p/n3
gamma0, gamma1 = 0.1 / np.sqrt(kappa), 0.1 / np.sqrt(kappa)
rho_12 = 0.2
sigma = 1

kappas = [kappa1, kappa2, kappa3]
ns = [n1, n2, n3]
rs = [r1, r2, r3]

# Compute Limit of Variance

In [5]:
# Solve system of equations for the three splits
alpha_i_star, sigma_i_star, lambda_i_star = np.ones(3), np.ones(3), np.ones(3)
for i in range(3):
    alpha_i_star[i], sigma_i_star[i], lambda_i_star[i] = solve_eqs(p/ns[i], gamma, ridge_param=0)

<ipython-input-2-0f9c5c810f10>:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
<ipython-input-3-ce0a7c9be852>:13: RuntimeWarning: invalid value encountered in sqrt
  density = lambda z1 ,z2 : np.multiply(1 / (2 * np.pi * np.sqrt(1 - rho ** 2)),np.exp(- (z1 ** 2 - np.multiply(2 * rho * z1,z2) + z2 ** 2) / 2 / (1 - rho ** 2)))
<ipython-input-3-ce0a7c9be852>:5: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  prox = lambda z : fsolve(lambda t  : lambda_ *  sigmoid(t) + t - z, z, xtol=1e-8)


In [6]:
# Compute the covariance matrix defined in Lemma D.6
cov_mat = [[gamma**2, alpha_i_star[0]*gamma**2, alpha_i_star[1]*gamma**2, alpha_i_star[2]*gamma**2], 
[alpha_i_star[0]*gamma**2, kappas[0] *sigma_i_star[0]**2 + alpha_i_star[0]**2*gamma**2, gamma**2*alpha_i_star[0]*alpha_i_star[1], gamma**2*alpha_i_star[0]*alpha_i_star[2]],
[alpha_i_star[1]*gamma**2, gamma**2*alpha_i_star[0]*alpha_i_star[1], kappas[1] *sigma_i_star[1]**2 + alpha_i_star[1]**2*gamma**2, gamma**2*alpha_i_star[1]*alpha_i_star[2]],
[alpha_i_star[2]*gamma**2, gamma**2*alpha_i_star[0]*alpha_i_star[2], gamma**2*alpha_i_star[1]*alpha_i_star[2], kappas[2] *sigma_i_star[2]**2 + alpha_i_star[2]**2*gamma**2]]
cov_mat = np.array(cov_mat)

In [7]:
# Compute e_{\gamma, C} where C = 0, defined in Appendix A.1, item 10
density = lambda z1 : 1/ np.sqrt(2 * np.pi) * np.exp(-(z1**2/2))
step_size = 0.025
z1 = np.arange(-8,8+step_size,step_size)
D = density(z1)
trapezoidal_rule = lambda X: step_size * sum(X.ravel())
integrands = z1 * sigmoid(gamma * z1) * D
e_gamma_0 = trapezoidal_rule(integrands)

In [8]:
# Compute h_i defined in Appendix A.1, item 10, s_i and t_i defined under Lemma A.1
ss, hs, ts = np.ones(3), np.ones(3), np.ones(3)
for i in range(3):
    curr_cov_mat = cov_mat[np.ix_([0,i+1],[0,i+1])]
    # square root of determinant of the covariance matrix
    det_root = np.sqrt(np.abs(np.linalg.det(curr_cov_mat)))
    # inverse of the covariance matrix
    inv_cov_mat = np.linalg.inv(curr_cov_mat)
    # density of the multivariate normal
    density = lambda z1 ,z2: 1 / (2 * np.pi * det_root) * np.exp(- 1/2* (z1**2*inv_cov_mat[0,0]  + 2*z1*z2*inv_cov_mat[0,1] + z2**2*inv_cov_mat[1,1]))

    # We now compute the density on a fine grid
    step_size = 0.025
    z1 = np.arange(-8,8+step_size,step_size)
    z2 = np.arange(-8,8+step_size,step_size)
    Z1,Z2 = np.meshgrid(z1,z2)
    D = density(Z1,Z2)

    # Helper functions for tensor and numerical integration
    make_tensor = lambda z1, z2: np.outer(z1,z2)
    trapezoidal_rule = lambda X: step_size ** 2 * sum(X.ravel())

    # Compute s_i
    t1 = 1/sigmoid(z2)
    t2 = sigmoid(z1)
    ss[i] = trapezoidal_rule(np.multiply(make_tensor(t1, t2),D))

    # Compute h_i and t_i
    t1 = 1/sigmoid(z2)
    t2 = z1 * sigmoid(z1)
    hs[i] = trapezoidal_rule(np.multiply(make_tensor(t1, t2),D))
    ts[i] = (rs[i]/2-p/n)*(1-4*e_gamma_0**2)

In [9]:
# Compute e_{gamma, -alpha_i_star * gamma} and q__{gamma, -alpha_i_star * gamma},
# which appear in the defintion of f_{i,j} under Lemma A.1
e_gamma_negative_s, q_gamma_negative_s = np.ones(3), np.ones(3)
for i in range(3):
    mu = - alpha_i_star[i] * gamma
    density = lambda z1: 1/ np.sqrt(2 * np.pi) * np.exp(-((z1 - mu)**2/2))
    step_size = 0.025
    z1 = np.arange(-8,8+step_size,step_size)
    D = density(z1)
    trapezoidal_rule = lambda X: step_size * sum(X.ravel())
    
    # Compute e_{\gamma, -alpha_i_star * gamma}
    integrands = z1 * sigmoid(gamma * z1) * D
    e_gamma_negative_s[i] = trapezoidal_rule(integrands)

    # Compute q_{\gamma, -alpha_i_star * gamma}
    integrands = sigmoid(gamma * z1) * D
    q_gamma_negative_s[i] = trapezoidal_rule(integrands)

In [10]:
# Compute f_{i,j} defined under Lemma A.1
fs = np.ones((3, 3))
for i in range(3):
    for j in range(3):
        t1 = 2 * rs[i] * hs[j] * e_gamma_0 / gamma
        expo_term = 0.5*(alpha_i_star[j]**2 * gamma**2 + kappas[j] * sigma_i_star[j]**2)
        t2 = 4 * p / n * e_gamma_0 * (e_gamma_0 + np.exp(expo_term) * e_gamma_negative_s[j])
        t3 = 2*p/n*(0.5 + np.exp(expo_term) * q_gamma_negative_s[j])
        fs[i,j] = t1 - t2 + t3

In [11]:
# Compute expectation of fractions
val_1_2_s = np.ones(3) # sigmoid(Z_beta) / sigmoid^2(Z_{beta_{s_i}})
val_2_1_s = np.ones(3) # sigmoid^2(Z_beta) / sigmoid(Z_{beta_{s_i}})
val_prime_1_s = np.ones(3) # sigmoid_prime(Z_beta) / sigmoid(Z_{beta_{s_i}})
val_0_i_over_i = np.ones(3) # sigmoid(Z_beta)Z_{beta_{s_i}} / sigmoid(Z_{beta_{s_i}})

for i in range(3):
    curr_cov_mat = cov_mat[np.ix_([0,i+1],[0,i+1])]
    det_root = np.sqrt(np.abs(np.linalg.det(curr_cov_mat)))
    inv_cov_mat = np.linalg.inv(curr_cov_mat)
    density = lambda z1, z2: 1 / (2 * np.pi * det_root) * np.exp(- 1/2* (z1**2*inv_cov_mat[0,0] + 2*z1*z2*inv_cov_mat[0,1] + z2**2*inv_cov_mat[1,1]))

    # We now compute the density on a fine grid
    step_size = 0.025
    z1 = np.arange(-8,8+step_size,step_size)
    z2 = np.arange(-8,8+step_size,step_size)
    Z1,Z2 = np.meshgrid(z1,z2)
    D = density(Z1,Z2)
    
    # Helper functions for tensor and numerical integration
    make_tensor = lambda z1  ,z2  : np.outer(z1,z2)
    trapezoidal_rule = lambda X  : step_size ** 2 * sum(X.ravel())

    # Compute the expectation of fractions
    t1 = 1/sigmoid(z2)**2
    t2 = sigmoid(z1)
    val_1_2_s[i] = trapezoidal_rule(np.multiply(make_tensor(t1, t2),D))

    t1 = 1/sigmoid(z2)
    t2 = sigmoid(z1)**2
    val_2_1_s[i] = trapezoidal_rule(np.multiply(make_tensor(t1, t2),D))

    t1 = 1/sigmoid(z2)
    t2 = sigmoid_prime(z1)
    val_prime_1_s[i] = trapezoidal_rule(np.multiply(make_tensor(t1, t2),D))

    t1 = z2/sigmoid(z2)
    t2 = sigmoid(z1)
    val_0_i_over_i[i] = trapezoidal_rule(np.multiply(make_tensor(t1, t2),D))


In [12]:
# proximal function
def prox(lambda_, z):
    return fsolve(lambda t: lambda_ *  sigmoid(t) + t - z, z, xtol=1e-8)

# Compute gs_{i, j} defined under Lemma A.1
gs_part1 = np.ones((3, 3))
gs_part2 = np.ones(3)
gs = np.ones((3, 3))

# Compute the first part of g_i 
for i in range(3):
    for j in range(3):
        if i == j:
            continue
        curr_cov_mat = cov_mat[np.ix_([0,i+1,j+1],[0,i+1,j+1])]
        det_root = np.sqrt(np.abs(np.linalg.det(curr_cov_mat)))
        inv_cov_mat = np.linalg.inv(curr_cov_mat)
        density = lambda z1 ,z2, z3 : 1 / ((2 * np.pi)**1.5 * det_root) *\
            np.exp(- 1/2* (z1**2*inv_cov_mat[0,0] + z2**2*inv_cov_mat[1,1] +\
            z3**2*inv_cov_mat[2,2] + 2*z1*z2*inv_cov_mat[0,1] + 2*z1*z3*inv_cov_mat[0,2]+\
            2*z2*z3*inv_cov_mat[1,2]))
        
        # We now compute the density on a fine grid
        step_size = 0.04
        z1 = np.arange(-8,8+step_size,step_size)
        z2 = np.arange(-8,8+step_size,step_size)
        z3 = np.arange(-8,8+step_size,step_size)
        Z1,Z2,Z3 = np.meshgrid(z1,z2,z3, indexing = 'ij')
        D = density(Z1,Z2,Z3)

        # Helper functions for tensor and numerical integration
        make_tensor = lambda z1, z2, z3  : np.einsum('i,j,k',z1,z2,z3)
        trapezoidal_rule = lambda X  : step_size ** 3 * sum(X.ravel())

        t1 = sigmoid(z1)
        t2 = 1/sigmoid(z2) - 1
        t3 = 1 - sigmoid(prox( lambda_i_star[j], z3 + lambda_i_star[j] ))
        gs_part1[i, j] = trapezoidal_rule(np.multiply(make_tensor(t1, t2, t3),D))	

# Compute the second part of g_i  
for i in range(3):
    curr_cov_mat = cov_mat[np.ix_([0,i+1],[0,i+1])]
    det_root = np.sqrt(np.abs(np.linalg.det(curr_cov_mat)))
    inv_cov_mat = np.linalg.inv(curr_cov_mat)
    density = lambda z1, z2: 1 / (2 * np.pi * det_root) * np.exp(- 1/2* (z1**2*inv_cov_mat[0,0] + 2*z1*z2*inv_cov_mat[0,1] + z2**2*inv_cov_mat[1,1]))

    # We now compute the density on a fine grid
    step_size = 0.025
    z1 = np.arange(-8,8+step_size,step_size)
    z2 = np.arange(-8,8+step_size,step_size)
    Z1,Z2 = np.meshgrid(z1,z2)
    D = density(Z1,Z2)
    
    # Helper functions for tensor and numerical integration
    make_tensor = lambda z1  ,z2  : np.outer(z1,z2)
    trapezoidal_rule = lambda X  : step_size ** 2 * sum(X.ravel())

    t1 = sigmoid(prox(lambda_i_star[i], z2))
    t2 = 1 - sigmoid(z1)
    gs_part2[i] = trapezoidal_rule(np.multiply(make_tensor(t1, t2),D))

# Add the two parts up
for i in range(3):
    for j in range(3):
        if i == j:
            continue
        gs[i, j] = gs_part1[i, j] + gs_part2[j]

<ipython-input-2-0f9c5c810f10>:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [13]:
# Compute expectation of more fractions
val_0_i_over_j = np.ones((3, 3)) # sigmoid(Z_beta) * Z_{beta_{s_i}} / sigmoid(Z_{beta_{s_j}})
val_0_over_ij = np.ones((3, 3)) # sigmoid(Z_beta) / sigmoid(Z_{beta_{s_i}}) / sigmoid(Z_{beta_{s_j}})

for i in range(3):
    for j in range(3):
        if i == j:
            continue
        curr_cov_mat = cov_mat[np.ix_([0,i+1,j+1],[0,i+1,j+1])]
        det_root = np.sqrt(np.abs(np.linalg.det(curr_cov_mat)))
        inv_cov_mat = np.linalg.inv(curr_cov_mat)
        density = lambda z1 ,z2, z3 : 1 / ((2 * np.pi)**1.5 * det_root) *\
            np.exp(- 1/2* (z1**2*inv_cov_mat[0,0] + z2**2*inv_cov_mat[1,1] +\
            z3**2*inv_cov_mat[2,2] + 2*z1*z2*inv_cov_mat[0,1] + 2*z1*z3*inv_cov_mat[0,2]+\
            2*z2*z3*inv_cov_mat[1,2]))
        
        # We now compute the density on a fine grid
        step_size = 0.04
        z1 = np.arange(-8,8+step_size,step_size)
        z2 = np.arange(-8,8+step_size,step_size)
        z3 = np.arange(-8,8+step_size,step_size)
        Z1,Z2,Z3 = np.meshgrid(z1,z2,z3, indexing = 'ij')
        D = density(Z1,Z2,Z3)

        # Helper functions for tensor and numerical integration
        make_tensor = lambda z1, z2, z3: np.einsum('i,j,k',z1,z2,z3)
        trapezoidal_rule = lambda X: step_size ** 3 * sum(X.ravel())

        t1 = sigmoid(z1)
        t2 = z2
        t3 = 1/sigmoid(z3)
        val_0_i_over_j[i, j] = trapezoidal_rule(np.multiply(make_tensor(t1, t2, t3),D))

        t1 = sigmoid(z1)
        t2 = 1/sigmoid(z2)
        t3 = 1/sigmoid(z3)
        val_0_over_ij[i, j] = trapezoidal_rule(np.multiply(make_tensor(t1, t2, t3),D))		


# Compute limit of terms listed in Lemma B.3

In [14]:
# Compute term (i)
var_t1 = 0
for s in permutations([0,1,2]):
	a,b,c = s[0], s[1], s[2]
	var_t1 += - 4 * e_gamma_0 * hs[a] / gamma / ts[b] * (ss[a] -1) + 4 * rs[c] * e_gamma_0**2 * hs[a]**2 / gamma**2 / rs[b] / ts[b]
	var_t1 += (ss[a]-1)**2 / ts[b] + 2 * kappa / (rs[b] - 2 * kappa) / rs[c] * (1 - 2 * ss[a] + val_1_2_s[a])
	term1 = gamma**2 * ((1 - alpha_i_star[a] )*ss[a] - val_2_1_s[a] + alpha_i_star[a]/2)**2
	term2 = kappas[a] * sigma_i_star[a]**2 * (ss[a]- 0.5)**2
	var_t1 += 2 / (rs[b] - 2 * kappa) * (term1 + term2)
var_t1 *= (sigma**2 / 18)

In [15]:
# Compute term (ii)
var_t2 = 0
for s in permutations([0,1,2]):
	a,b,c = s[0], s[1], s[2]
	var_t2 += val_1_2_s[a] / rs[c]
var_t2 *= (sigma**2 / 18)

In [16]:
# Compute term (iii)
cov_t1 = 0
for s in permutations([0,1,2]):
	a,b,c = s[0], s[1], s[2]
	cov_t1 += ss[a]*(ss[a]-1)/ts[b] - 2 * e_gamma_0 * hs[a] * ss[a] / gamma / ts[b] + 2 / rs[b] * (val_prime_1_s[a] * hs[a] - (ss[a]-0.5)*val_0_i_over_i[a]  )
	cov_t1 += (2 * e_gamma_0 * hs[a] / gamma - ss[a] + 1) * fs[b,a] / rs[b] / ts[b]
cov_t1 *= ((-2) * sigma**2 / 18)

In [17]:
# Compute term (iv)
cov_t2 = 0
for s in permutations([0,1,2]):
	a,b,c = s[0], s[1], s[2]
	cov_t2 += ss[a]*(ss[a]-1)/ts[b] - 2 * e_gamma_0 * hs[a] * ss[a] / gamma / ts[b] + 2 / rs[b] * val_prime_1_s[c] * hs[a]   
	cov_t2 += (2 * e_gamma_0 * hs[a] / gamma - ss[a] + 1) * fs[b, c] / rs[b] / ts[b]
	cov_t2 += - 2 * (ss[c] - 0.5)/rs[b] * (lambda_i_star[c]*gs[a,c] + val_0_i_over_j[c, a])
cov_t2 *= ((-2) * sigma**2 / 18)

In [18]:
# Compute term (v)
cov_t3 = 0
for s in permutations([0,1,2]):
	a,b,c = s[0], s[1], s[2]
	cov_t3 += val_0_over_ij[b, c] /rs[a]
cov_t3 *= (sigma**2 / 18)

In [19]:
# Compute term (vi)
cov_t4 = 0
for s in permutations([0,1,2]):
	a,b,c = s[0], s[1], s[2]
	cov_t4 += (ss[a]-1)*(ss[c]-1)/ts[b] - 2*hs[c]*e_gamma_0*(ss[a]-1)/gamma/ts[b] - 2*hs[a]*e_gamma_0*(ss[c]-1)/gamma/ts[b]
	cov_t4 += 4 * np.sqrt(rs[a] * rs[c]) * hs[a] * hs[c] * e_gamma_0**2 / rs[b] / gamma**2 /ts[b]
	term1 = val_prime_1_s[a] - alpha_i_star[a]*(ss[a]-0.5)
	term2 = val_prime_1_s[c] - alpha_i_star[c]*(ss[c]-0.5)
	cov_t4 += 2*gamma**2/(rs[b] - 2*kappa)*term1*term2
	cov_t4 += - 2 * lambda_i_star[c] / (rs[b] - 2 * kappa)*gs[a,c]*(ss[c]-0.5)
	cov_t4 += - 2 * lambda_i_star[a] / (rs[b] - 2 * kappa)*gs[c,a]*(ss[a]-0.5)
cov_t4 *= (sigma**2 / 18)

# Compute the limit of the cross-fitting estimator

In [20]:
overall_var = var_t1 + var_t2 +  cov_t1 + cov_t2 + cov_t3 + cov_t4
overall_var += (kappa * (gamma0**2 + gamma1**2 - 2 * rho_12 * gamma0 * gamma1) * (1/r1 + 1/r2 + 1/r3) / 9)
print(f"Asymptotic variance of the cross-fitting estimator is: {overall_var}")

Asymptotic variance of the cross-fitting estimator is: 29.18999691137837


In [21]:
classical_var = 2 * sigma**2 * (1 + np.exp(gamma**2/2)) + kappa * gamma1**2 + kappa * gamma0**2 - 2 * kappa * rho_12 * gamma1*gamma0
print(f"According to the classical formula, the asymptotic variance of the cross-fitting estimator is: {classical_var}")

According to the classical formula, the asymptotic variance of the cross-fitting estimator is: 4.0260250417188015
